In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain

In [ ]:
loader = PyPDFLoader("../données/Covid/EpidemioMain.pdf")
docs = loader.load()

In [ ]:
llm = ChatOllama(model='llama4:latest')

In [ ]:
prompt = PromptTemplate.from_template(
    "Tu dois répondre à la question en te basant sur les documents suivants :\n\n{context}\n\nQuestion: {question}"
)

In [ ]:
chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

In [ ]:
response =  chain.invoke({
    "context": docs,
    "question": "De quoi parle ce document ?"
})

In [ ]:
print(response)